In [ ]:
import os
import pandas as pd
import numpy as np
#import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle
import gzip as gz

import sys
sys.path.insert(0, '/home/jw3514/Work/ASD_Circuits_CellType/src/')
from ASD_Circuits import *
#from CellType_PSY import *
os.chdir("/home/jw3514/Work/ASD_Circuits_CellType/notebooks_mouse_sc/")
print(f"Current working directory: {os.getcwd()}")

#  Figure 5 (Cell type results)

In [ ]:
def FixSubiculum(DF):
    X = DF.loc["Subiculum_dorsal_part"]
    Y = DF.loc["Subiculum_ventral_part"]
    Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
    DF.loc["Subiculum"] = Z
    DF = DF.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])
    return DF

In [ ]:
ASD_STR_Bias = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col=0)
ASD_STR_Bias = FixSubiculum(ASD_STR_Bias)
ASD_CircuitsSet = pd.read_csv(
    "/home/jw3514/Work/ASD_Circuits/notebooks/ASD.SA.Circuits.Size46.csv",
    index_col="idx")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
ASD_Circuits.append("Subiculum")

## 5A STR Correlation

In [ ]:
ASD_STR_Bias = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col=0)
ASD_STR_Bias = FixSubiculum(ASD_STR_Bias)
#SC_Agg_Bias = pd.read_csv("dat/JonDat/MERFISH_Bias_NeuroOnly_JC.csv", index_col=0)
SC_Agg_Bias = pd.read_csv("dat/Bias/STR/ASD.MERFISH_Allen.NM.ISHMatch.Z2.csv", index_col=0)

In [ ]:
#SC_Agg_Bias = SC_Agg_Bias.rename(columns={"RANK":"Rank"})

In [ ]:
ASD_STR_Bias['REGION'] = ASD_STR_Bias['REGION'].replace('Amygdalar', 'Amygdala')

In [ ]:
ASD_CircuitsSet = pd.read_csv(
    "/home/jw3514/Work/ASD_Circuits/notebooks/ASD.SA.Circuits.Size46.csv",
    index_col="idx")
ASD_Circuits = ASD_CircuitsSet.loc[3, "STRs"].split(";")
ASD_Circuits.append("Subiculum")

In [ ]:
for i,row in ASD_STR_Bias.iterrows():
    ASD_STR_Bias.loc[i, "SC_Bias"] = SC_Agg_Bias.loc[i, "EFFECT"]
    ASD_STR_Bias.loc[i, "SC_Rank"] = int(SC_Agg_Bias.loc[i, "Rank"])
    if i in ASD_Circuits:
        ASD_STR_Bias.loc[i, "isCir"] = 1
    else:
        ASD_STR_Bias.loc[i, "isCir"] = 0

In [ ]:
def TestBiasCompare(Bias1, Bias2, name1="1", name2="2"):
    REGIONS = list(set(Bias1["REGION"].values))

    REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 
                   'Hippocampus','Amygdala','Striatum', 
                   "Thalamus", "Hypothalamus", "Midbrain", 
                   "Medulla", "Pallidum", "Pons", 
                   "Cerebellum"]
    REG_COR_Dic = dict(zip(REGIONS_seq, ["#268ad5", "#D5DBDB", "#7ac3fa", 
                                     "#2c9d39", "#742eb5", "#ed8921", 
                                     "#e82315", "#E6B0AA", "#f6b26b",  
                                     "#20124d", "#2ECC71", "#D2B4DE", 
                                     "#ffd966", ]))
    plt.figure(dpi=300, figsize=(6, 4))

    # Use Seaborn for style and color palette
    sns.set(style="whitegrid", context="talk")
    palette = sns.color_palette("deep")

    JoinDF = Bias1.join(Bias2, how="inner", lsuffix="_{}".format(name1), rsuffix="_{}".format(name2))
    R,P = pearsonr(JoinDF["EFFECT_{}".format(name1)].values, JoinDF["EFFECT_{}".format(name2)].values)
    RR,PP = spearmanr(JoinDF["EFFECT_{}".format(name1)].values, JoinDF["EFFECT_{}".format(name2)].values)
    # Scatter plot with improved aesthetics
    for reg in REGIONS_seq:
        RegDF = JoinDF[JoinDF["REGION_{}".format(name1)]==reg]
        X = RegDF["EFFECT_{}".format(name1)].values
        Y = RegDF["EFFECT_{}".format(name2)].values
        plt.scatter(X, Y, facecolor=REG_COR_Dic[reg], alpha=0.7, s=20, label=reg.replace("_", " "))
    minX, maxY = min(JoinDF["EFFECT_{}".format(name1)].values), max(JoinDF["EFFECT_{}".format(name2)].values)
    
    Top50_X = JoinDF.sort_values("EFFECT_{}".format(name1), ascending=False).head(50).index.values
    top50_Y = JoinDF.sort_values("EFFECT_{}".format(name2), ascending=False).head(50).index.values 
    STR_Com = len(set(Top50_X).intersection(set(top50_Y)))
    #plt.text(x=minX, y=maxY*0.7, s="r=%.2f P<%.1e\nrho=%.2f P<%.1e\nSTR_Comm=%d"%(R, P, RR, PP, STR_Com), 
    #         fontsize=6, weight='bold')
    #plt.text(x=minX, y=maxY*0.7, s="r=%.2f\nP<%.1e\nrho=%.2f P<%.1e"%(R, np.max([P, 1e-10]), RR, 
    #                                                                 np.max([PP, 1e-10])), 
    #         fontsize=6, weight='bold')
    plt.text(x=minX-0.2, y=maxY*0.7, s="r=%.2f\nP<%.1e"%(R, np.max([P, 1e-10])), 
             fontsize=12, weight='bold')
    # Adding labels, title, and legend
    plt.xlabel(name1, fontsize=14, weight='bold')
    plt.ylabel(name2, fontsize=14, weight='bold')
    #plt.title("Scatter Plot of EFFECT vs. SC_Bias", fontsize=16, weight='bold')
    #plt.legend(fontsize=6)
    plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.plot([-1.1, 0.8], [-1.1, 0.8], color="grey", alpha=0.3)
    plt.xlim(-1.1, 0.8)
    plt.ylim(-1.1, 0.8)
    # Adding grid lines
    plt.grid(True, linestyle='--', alpha=0.5)

    # Adjusting ticks
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
Bias1 = ASD_STR_Bias
Bias2 = SC_Agg_Bias
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Bias", "MERFISH Bias")

In [ ]:
REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 
               'Hippocampus','Amygdala','Striatum', 
               "Thalamus", "Hypothalamus", "Midbrain", 
               "Medulla", "Pallidum", "Pons", 
               "Cerebellum"]
REG_COR_Dic = dict(zip(REGIONS_seq, ["#268ad5", "#D5DBDB", "#7ac3fa", 
                                 "#2c9d39", "#742eb5", "#ed8921", 
                                 "#e82315", "#E6B0AA", "#f6b26b",  
                                 "#20124d", "#2ECC71", "#D2B4DE", 
                                 "#ffd966", ]))

In [ ]:
ISH_REG_Bias = []
MF_REG_Bias = []
for Reg in REGIONS_seq:
    ISH_Reg = ASD_STR_Bias[ASD_STR_Bias["REGION"]==Reg]["EFFECT"]
    MF_Reg = SC_Agg_Bias[SC_Agg_Bias["REGION"]==Reg]["EFFECT"]
    #print(Reg, ISH_Reg.max() - ISH_Reg.min(), MF_Reg.max() - MF_Reg.min())
    #ISH_REG_Bias.append(ISH_Reg.mean())
    #MF_REG_Bias.append(MF_Reg.mean())
    ISH_REG_Bias.append(np.nanmean(ISH_Reg))
    MF_REG_Bias.append(np.nanmean(MF_Reg))
    print(Reg, np.nanmean(ISH_Reg), np.nanmean(MF_Reg))

In [ ]:
R1, P1 = pearsonr(ISH_REG_Bias, MF_REG_Bias)
R2, P2 = spearmanr(ISH_REG_Bias, MF_REG_Bias)
print(R1, P1, R2, P2)
plt.figure(dpi=200, figsize=(6,6))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_STR_Bias[ASD_STR_Bias["REGION"]==Reg]["EFFECT"]
    MF_Reg = SC_Agg_Bias[SC_Agg_Bias["REGION"]==Reg]["EFFECT"]
    X = ISH_Reg.mean()
    Y = MF_Reg.mean()
    plt.scatter(ISH_Reg.mean(), MF_Reg.mean(), color=REG_COR_Dic[Reg])
    plt.text(X, Y, s=Reg, fontsize=15)
plt.plot([-0.6, 0.5], [-0.6, 0.5], color="grey", alpha=0.3)
plt.text(-0.4, 0.2, s="R = %.2f\nrho = %.2f"%(R1, R2), fontsize=15)
plt.xlabel("ISH")
plt.xlim(-0.6, 0.5)
plt.ylim(-0.6, 0.5)
plt.ylabel("MERFISH per Vol")

## 5B Sub Types

In [ ]:
#ASD_Cluster_Bias_for_DN = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.pvalues.csv", index_col=0)
#ASD_Cluster_Bias_noDN = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.noDN.pvalues.csv", index_col=0)
ASD_BiasDN_z1clip3_addP = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.z1clip3.addP.csv", index_col=0)
#ASD_Bias.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.pvalues.csv")

### V1 QQ Plot

In [ ]:
def GetExpQ(pvalues):
    sorted_pvalues = np.sort(pvalues)
    expected = np.linspace(0, 1, len(sorted_pvalues), endpoint=False)[1:]
    expected_quantiles = -np.log10(expected)
    observed_quantiles = -np.log10(sorted_pvalues[1:])
    return expected_quantiles, observed_quantiles

In [ ]:
def QQPlot_MouseSC(DF_w_Pvalues, Classes, max_val=4.1):
    P_Q005_DF = DF_w_Pvalues[DF_w_Pvalues["qvalues"]<0.05]#
    if P_Q005_DF.shape[0] > 0:
        P_Q005 = P_Q005_DF["Pvalue"][-1]
    else:
        P_Q005 = -1
    P_Q01_DF = DF_w_Pvalues[DF_w_Pvalues["qvalues"]<0.1]#["Pvalue"][-1]
    if P_Q01_DF.shape[0] > 0:
        P_Q01 = P_Q01_DF["Pvalue"][-1]
    else:
        P_Q01 = -1

    plt.figure(dpi=300, figsize=(8, 8))
    sns.set(style="whitegrid", context="talk")

    # Plot the quantiles
    for Class in Classes:
        tmp = DF_w_Pvalues[DF_w_Pvalues["class_id_label"]==Class]
        Class_Pvals = tmp["Pvalue"].values
        Qexp, Qobs = GetExpQ(Class_Pvals)
        plt.scatter(Qexp, Qobs , alpha=0.7, s=50, label=Class)

    Other = DF_w_Pvalues[~DF_w_Pvalues["class_id_label"].isin(Classes)]
    Other_Pvals = Other["Pvalue"].values
    Qexp, Qobs = GetExpQ(Other_Pvals)
    plt.scatter(Qexp, Qobs , alpha=0.7, color="grey", s=50, label="Other")

    if P_Q005 != -1:
        plt.axhline(y=-np.log10(P_Q005), color='grey', linestyle='--', linewidth=2,)
        plt.text(x=3.25, y=-np.log10(P_Q005) + 0.05, s="FDR 0.05")
    if P_Q01 != -1:
        plt.axhline(y=-np.log10(P_Q01), color='grey', linestyle='--', linewidth=2,)
        plt.text(x=3.25, y=-np.log10(P_Q01) + 0.05, s="FDR 0.1")

    # Add a reference line
    plt.plot([0, max_val], [0, max_val], 'r--', linewidth=2)

    # Add labels and title
    plt.xlabel('Expected -log10(p-value)', fontsize=25, weight='bold')
    plt.ylabel('Observed -log10(p-value)', fontsize=25, weight='bold')
    #plt.title(title, fontsize=16, weight='bold')

    # Customize the ticks
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    # Add grid
    plt.grid(True, linestyle='--', alpha=0.5)

    # Tight layout
    plt.tight_layout()
    plt.legend(fontsize=15, loc="lower right")

    # Show the plot
    plt.show()

In [ ]:
Classes = ["01 IT-ET Glut",  "02 NP-CT-L6b Glut", 
           "09 CNU-LGE GABA", "18 TH Glut", "11 CNU-HYa GABA", "13 CNU-HYa Glut"]

In [ ]:
QQPlot_MouseSC(ASD_BiasDN_z1clip3_addP, Classes)

## V2 Boxplot

In [ ]:
CellTypesDF = pd.read_csv("dat/CellTypeHierarchy.csv")
Class2Cluster = {}
Subclass2Cluster = {}
for i, row in CellTypesDF.iterrows():
    _cluster, _class, _subclass, _supertype = row
    if _class not in Class2Cluster:
        Class2Cluster[_class] = []
    if _subclass not in Subclass2Cluster:
        Subclass2Cluster[_subclass] = []
    Class2Cluster[_class].append(_cluster)
    Subclass2Cluster[_subclass].append(_cluster)

In [ ]:
def ClassBiasBoxPlot(DF):
    Class = sorted(Class2Cluster.keys())
    sns.set(style="whitegrid", context="talk")
    # Calculate medians and sort data by medians
    ASD_dat_CB = []
    medians = []
    for _CT in Class:
        subdf = DF[DF["class_id_label"]==_CT]
        data = [x for x in subdf["EFFECT"].values if x==x]
        ASD_dat_CB.append(data)
        medians.append(np.median(data))

    # Sort based on medians
    sorted_indices = np.argsort(medians)
    sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
    sorted_Class = [Class[i] for i in sorted_indices]

    # Plotting
    fig, ax = plt.subplots(dpi=480, figsize=(8, 8))
    boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)
    colors = sns.color_palette("muted", len(sorted_Class))
    for patch, color in zip(boxplot['boxes'], colors):
        patch.set_facecolor(color)

    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xlabel('ASD Bias', fontsize=14)
    plt.ylabel('Classes', fontsize=14)
    plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')
    ax.grid(True, linestyle='--', alpha=0.7)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
ClassBiasBoxPlot(ASD_BiasDN_z1clip3_addP)

# C - Subclass level for selected classes

In [ ]:
# Sample data generation (using the provided example structure)
Subclass = sorted(Subclass2Cluster.keys())
CTX_Glut = Subclass[0:34] # CTX Glut
dat = []
for _CT in CTX_Glut:
    clusters_ = Subclass2Cluster[_CT]
    clusters_ = [x for x in clusters_ if x in ASD_BiasDN_z1clip3_addP.index.values]
    dat.append([x for x in ASD_BiasDN_z1clip3_addP.loc[clusters_, "EFFECT"].values if x==x])

# Calculate medians and sort data based on medians
medians = [np.median(d) for d in dat]
sorted_indices = np.argsort(medians)[::-1]
sorted_dat = [dat[i] for i in sorted_indices]
sorted_CTX_Glut = [CTX_Glut[i] for i in sorted_indices]

# Set up the plotting style and context
sns.set(style="whitegrid", context="talk")

# Create the plot
fig, ax = plt.subplots(dpi=480, figsize=(8, 9))  # Adjust the figure size as needed

# Create the boxplot
sns.boxplot(data=sorted_dat, orient='h', ax=ax, palette='deep')

# Set the labels
ax.set_xlabel("ASD Bias", fontsize=16, weight='bold')
ax.set_yticklabels(sorted_CNU_HYa_Glut, fontsize=12)

# Improve grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Customize the ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)

# Add title if necessary
# plt.title("ASD Bias Across Different Subclasses", fontsize=18, weight='bold')

# Display the plot
plt.show()

In [ ]:
test = ['006 L4/5 IT CTX Glut', '007 L2/3 IT CTX Glut']
sorted_CTX_Glut

In [ ]:
# Statistical test for enrichment at top of ranked list
# Testing if 2 specific neuron types appear at the top of sorted_CTX_Glut

from scipy.stats import hypergeom
from scipy.stats import rankdata
import numpy as np

# Your test types
test = ['006 L4/5 IT CTX Glut', '007 L2/3 IT CTX Glut']

# Check if sorted_CTX_Glut exists and get its length
if 'sorted_CTX_Glut' in locals():
    N = len(sorted_CTX_Glut)  # Total number of CTX_Glut types
    M = len(test)  # Number of test types (2)
    
    # Find positions of test types in sorted list (1-indexed for ranks)
    test_positions = []
    for t in test:
        if t in sorted_CTX_Glut:
            pos = sorted_CTX_Glut.index(t) + 1  # 1-indexed rank
            test_positions.append(pos)
    
    print(f"Total CTX_Glut types: {N}")
    print(f"Test types: {test}")
    print(f"Positions in ranked list: {test_positions}")
    print(f"Mean rank: {np.mean(test_positions):.2f}")
    
    # ===== OPTION 1: HYPERGEOMETRIC TEST =====
    # Tests: "Are the test types overrepresented in top K positions?"
    # Best when: You have a specific cutoff K (e.g., top 5, top 10)
    
    print("\n" + "="*60)
    print("HYPERGEOMETRIC TEST")
    print("="*60)
    
    # Test multiple K values (top K positions)
    for K in [2, 3, 5, 10]:
        # Count how many test types are in top K
        observed = sum(1 for pos in test_positions if pos <= K)
        
        # Hypergeometric test: P(X >= observed)
        # Parameters: N (population), M (successes in population), K (sample size)
        p_value = hypergeom.sf(observed - 1, N, M, K)  # sf = survival function (1 - cdf)
        
        print(f"\nTop {K} positions:")
        print(f"  Observed test types in top {K}: {observed}/{M}")
        print(f"  Expected by chance: {M * K / N:.2f}")
        print(f"  P-value (one-tailed): {p_value:.4e}")
        if p_value < 0.05:
            print(f"  *** Significant enrichment! ***")
    
    # ===== OPTION 2: PERMUTATION TEST =====
    # Tests: "Do the test types have unusually high ranks?"
    # Best when: You want to test rank-based statistics without fixed cutoff
    
    print("\n" + "="*60)
    print("PERMUTATION TEST")
    print("="*60)
    
    n_permutations = 10000
    observed_mean_rank = np.mean(test_positions)
    
    # Generate null distribution by permuting ranks
    null_mean_ranks = []
    for _ in range(n_permutations):
        # Randomly select 2 positions from 1 to N
        random_positions = np.random.choice(range(1, N+1), size=M, replace=False)
        null_mean_ranks.append(np.mean(random_positions))
    
    null_mean_ranks = np.array(null_mean_ranks)
    
    # Calculate p-value: proportion of permutations with mean rank <= observed
    p_value_perm = np.mean(null_mean_ranks <= observed_mean_rank)
    
    print(f"\nObserved mean rank: {observed_mean_rank:.2f}")
    print(f"Expected mean rank (null): {np.mean(null_mean_ranks):.2f}")
    print(f"P-value (one-tailed): {p_value_perm:.4e}")
    if p_value_perm < 0.05:
        print(f"  *** Significant enrichment! ***")
    
    # ===== SUM OF RANKS TEST =====
    # Tests: "Is the sum of ranks lower than expected?" (lower sum = better ranks)
    # This is equivalent to mean rank but sometimes more intuitive
    
    observed_sum_rank = np.sum(test_positions)
    
    # Generate null distribution for sum of ranks
    null_sum_ranks = []
    for _ in range(n_permutations):
        random_positions = np.random.choice(range(1, N+1), size=M, replace=False)
        null_sum_ranks.append(np.sum(random_positions))
    
    null_sum_ranks = np.array(null_sum_ranks)
    
    # Calculate p-value: proportion of permutations with sum rank <= observed
    # Lower sum = better (both types higher up), so we test P(sum <= observed)
    p_value_sum = np.mean(null_sum_ranks <= observed_sum_rank)
    
    print(f"\n" + "-"*60)
    print(f"SUM OF RANKS TEST")
    print(f"-"*60)
    print(f"Observed sum of ranks: {observed_sum_rank}")
    print(f"Expected sum of ranks (null): {np.mean(null_sum_ranks):.2f}")
    print(f"P-value (one-tailed, testing if sum <= observed): {p_value_sum:.4e}")
    if p_value_sum < 0.05:
        print(f"  *** Significant enrichment! ***")
    print(f"\nNote: Lower sum = better ranks. Testing if sum_rank <= observed_sum_rank")
    print(f"      This is equivalent to testing mean rank, but sum can be more intuitive.")
    
    # Also test: Are both types in top K? (more stringent)
    print(f"\nTesting if BOTH types are in top positions:")
    for K in [2, 3, 5]:
        both_in_top = all(pos <= K for pos in test_positions)
        # Permutation test: probability that 2 random types both fall in top K
        null_both_in_top = []
        for _ in range(n_permutations):
            random_positions = np.random.choice(range(1, N+1), size=M, replace=False)
            null_both_in_top.append(all(pos <= K for pos in random_positions))
        p_both = np.mean(null_both_in_top)
        
        print(f"  Top {K}: Both in top? {both_in_top}, P-value: {p_both:.4e}")
    
    print("\n" + "="*60)
    print("RECOMMENDATION:")
    print("="*60)
    print("""
    Use HYPERGEOMETRIC TEST if:
    - You have a specific biological cutoff (e.g., "top 5 most biased types")
    - You want exact p-values
    - You're testing enrichment in a specific region of the ranking
    
    Use PERMUTATION TEST if:
    - You want to test overall rank enrichment without fixed cutoff
    - You want more flexibility in test statistics
    - You're comfortable with approximate p-values
    
    For your case (testing if 2 types are at the "top"), I recommend:
    1. Hypergeometric test with K=5 or K=10 (most interpretable)
    2. Permutation test for mean rank (more flexible, no cutoff needed)
    """)
    
else:
    print("Error: sorted_CTX_Glut not found. Please run the cell that creates it first.")


In [ ]:
0.02 * 0.1

In [ ]:
# Sample data generation (using the provided example structure)
Subclass = sorted(Subclass2Cluster.keys())
CTX_GABA = Subclass[45:58] + Subclass[66:72]  # CTX GABA
dat = []
for _CT in CTX_GABA:
    dat.append([x for x in ASD_BiasDN_z1clip3_addP.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])

# Calculate medians and sort data based on medians
medians = [np.median(d) for d in dat]
sorted_indices = np.argsort(medians)[::-1]
sorted_dat = [dat[i] for i in sorted_indices]
sorted_CTX_GABA = [CTX_GABA[i] for i in sorted_indices]

# Set up the plotting style and context
sns.set(style="whitegrid", context="talk")

# Create the plot
fig, ax = plt.subplots(dpi=480, figsize=(8, 6))  # Adjust the figure size as needed

# Create the boxplot
sns.boxplot(data=sorted_dat, orient='h', ax=ax, palette='deep')

# Set the labels
ax.set_xlabel("ASD Bias", fontsize=16, weight='bold')
ax.set_yticklabels(sorted_CTX_GABA, fontsize=12)

# Improve grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Customize the ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)

# Add title if necessary
# plt.title("ASD Bias Across Different Subclasses", fontsize=18, weight='bold')

# Display the plot
plt.show()

In [ ]:
CTX_GABA

In [ ]:
# Statistical test for enrichment at top of ranked list - CTX_GABA
# Testing if 2 specific neuron types appear at the top of sorted_CTX_GABA

from scipy.stats import hypergeom
from scipy.stats import rankdata
import numpy as np

# Your test types for CTX_GABA
test_GABA = ['046 Vip Gaba', '053 Sst Gaba']

# Check if sorted_CTX_GABA exists and get its length
if 'sorted_CTX_GABA' in locals() or 'sorted_CTX_GABA' in globals():
    N = len(sorted_CTX_GABA)  # Total number of CTX_GABA types
    M = len(test_GABA)  # Number of test types (2)
    
    # Find positions of test types in sorted list (1-indexed for ranks)
    test_positions = []
    for t in test_GABA:
        if t in sorted_CTX_GABA:
            pos = sorted_CTX_GABA.index(t) + 1  # 1-indexed rank
            test_positions.append(pos)
        else:
            print(f"Warning: {t} not found in sorted_CTX_GABA")
    
    if len(test_positions) == M:
        print(f"Total CTX_GABA types: {N}")
        print(f"Test types: {test_GABA}")
        print(f"Positions in ranked list: {test_positions}")
        print(f"Mean rank: {np.mean(test_positions):.2f}")
        
        # ===== OPTION 1: HYPERGEOMETRIC TEST =====
        # Tests: "Are the test types overrepresented in top K positions?"
        # Best when: You have a specific cutoff K (e.g., top 5, top 10)
        
        print("\n" + "="*60)
        print("HYPERGEOMETRIC TEST - CTX_GABA")
        print("="*60)
        
        # Test multiple K values (top K positions)
        for K in [2, 3, 5, 10]:
            # Count how many test types are in top K
            observed = sum(1 for pos in test_positions if pos <= K)
            
            # Hypergeometric test: P(X >= observed)
            # Parameters: N (population), M (successes in population), K (sample size)
            p_value = hypergeom.sf(observed - 1, N, M, K)  # sf = survival function (1 - cdf)
            
            print(f"\nTop {K} positions:")
            print(f"  Observed test types in top {K}: {observed}/{M}")
            print(f"  Expected by chance: {M * K / N:.2f}")
            print(f"  P-value (one-tailed): {p_value:.4e}")
            if p_value < 0.05:
                print(f"  *** Significant enrichment! ***")
        
        # ===== OPTION 2: PERMUTATION TEST =====
        # Tests: "Do the test types have unusually high ranks?"
        # Best when: You want to test rank-based statistics without fixed cutoff
        
        print("\n" + "="*60)
        print("PERMUTATION TEST - CTX_GABA")
        print("="*60)
        
        n_permutations = 10000
        observed_mean_rank = np.mean(test_positions)
        
        # Generate null distribution by permuting ranks
        null_mean_ranks = []
        for _ in range(n_permutations):
            # Randomly select 2 positions from 1 to N
            random_positions = np.random.choice(range(1, N+1), size=M, replace=False)
            null_mean_ranks.append(np.mean(random_positions))
        
        null_mean_ranks = np.array(null_mean_ranks)
        
        # Calculate p-value: proportion of permutations with mean rank <= observed
        p_value_perm = np.mean(null_mean_ranks <= observed_mean_rank)
        
        print(f"\nObserved mean rank: {observed_mean_rank:.2f}")
        print(f"Expected mean rank (null): {np.mean(null_mean_ranks):.2f}")
        print(f"P-value (one-tailed): {p_value_perm:.4e}")
        if p_value_perm < 0.05:
            print(f"  *** Significant enrichment! ***")
        
        # ===== SUM OF RANKS TEST =====
        # Tests: "Is the sum of ranks lower than expected?" (lower sum = better ranks)
        # This is equivalent to mean rank but sometimes more intuitive
        
        observed_sum_rank = np.sum(test_positions)
        
        # Generate null distribution for sum of ranks
        null_sum_ranks = []
        for _ in range(n_permutations):
            random_positions = np.random.choice(range(1, N+1), size=M, replace=False)
            null_sum_ranks.append(np.sum(random_positions))
        
        null_sum_ranks = np.array(null_sum_ranks)
        
        # Calculate p-value: proportion of permutations with sum rank <= observed
        # Lower sum = better (both types higher up), so we test P(sum <= observed)
        p_value_sum = np.mean(null_sum_ranks <= observed_sum_rank)
        
        print(f"\n" + "-"*60)
        print(f"SUM OF RANKS TEST - CTX_GABA")
        print(f"-"*60)
        print(f"Observed sum of ranks: {observed_sum_rank}")
        print(f"Expected sum of ranks (null): {np.mean(null_sum_ranks):.2f}")
        print(f"P-value (one-tailed, testing if sum <= observed): {p_value_sum:.4e}")
        if p_value_sum < 0.05:
            print(f"  *** Significant enrichment! ***")
        print(f"\nNote: Lower sum = better ranks. Testing if sum_rank <= observed_sum_rank")
        print(f"      This is equivalent to testing mean rank, but sum can be more intuitive.")
        
        # Also test: Are both types in top K? (more stringent)
        print(f"\nTesting if BOTH types are in top positions:")
        for K in [2, 3, 5]:
            both_in_top = all(pos <= K for pos in test_positions)
            # Permutation test: probability that 2 random types both fall in top K
            null_both_in_top = []
            for _ in range(n_permutations):
                random_positions = np.random.choice(range(1, N+1), size=M, replace=False)
                null_both_in_top.append(all(pos <= K for pos in random_positions))
            p_both = np.mean(null_both_in_top)
            
            print(f"  Top {K}: Both in top? {both_in_top}, P-value: {p_both:.4e}")
        
        print("\n" + "="*60)
        print("RECOMMENDATION:")
        print("="*60)
        print("""
        Use HYPERGEOMETRIC TEST if:
        - You have a specific biological cutoff (e.g., "top 5 most biased types")
        - You want exact p-values
        - You're testing enrichment in a specific region of the ranking
        
        Use PERMUTATION TEST if:
        - You want to test overall rank enrichment without fixed cutoff
        - You want more flexibility in test statistics
        - You're comfortable with approximate p-values
        
        For your case (testing if 2 types are at the "top"), I recommend:
        1. Hypergeometric test with K=5 or K=10 (most interpretable)
        2. Permutation test for mean rank (more flexible, no cutoff needed)
        """)
    else:
        print(f"Error: Only found {len(test_positions)}/{M} test types in sorted_CTX_GABA")
        print(f"Found positions: {test_positions}")
        
else:
    print("Error: sorted_CTX_GABA not found. Please run the cell that creates it first.")


In [ ]:
1.1820e-01 * 2.2000e-02

In [ ]:
# Sample data generation (using the provided example structure)
Subclass = sorted(Subclass2Cluster.keys())
CNU_HYa_Glut = Subclass[58:65]+Subclass[36:45]
dat = []
for _CT in CNU_HYa_Glut:
    dat.append([x for x in ASD_BiasDN_z1clip3_addP.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])

# Calculate medians and sort data based on medians
medians = [np.median(d) for d in dat]
sorted_indices = np.argsort(medians)[::-1]
sorted_dat = [dat[i] for i in sorted_indices]
sorted_CNU_HYa_Glut = [CNU_HYa_Glut[i] for i in sorted_indices]

# Set up the plotting style and context
sns.set(style="whitegrid", context="talk")

# Create the plot
fig, ax = plt.subplots(dpi=480, figsize=(7, 4))  # Adjust the figure size as needed

# Create the boxplot
sns.boxplot(data=sorted_dat, orient='h', ax=ax, palette='deep')

# Set the labels
ax.set_xlabel("ASD Bias", fontsize=16, weight='bold')
ax.set_yticklabels(sorted_CNU_HYa_Glut, fontsize=12)

# Improve grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Customize the ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
# Sample data generation (using the provided example structure)
Subclass = sorted(Subclass2Cluster.keys())
CNU_HYa_Glut = Subclass[73:91] + Subclass[110:122]
dat = []
for _CT in CNU_HYa_Glut:
    dat.append([x for x in ASD_BiasDN_z1clip3_addP.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])

# Calculate medians and sort data based on medians
medians = [np.median(d) for d in dat]
sorted_indices = np.argsort(medians)[::-1]
sorted_dat = [dat[i] for i in sorted_indices]
sorted_CNU_HYa_Glut = [CNU_HYa_Glut[i] for i in sorted_indices]

# Set up the plotting style and context
sns.set(style="whitegrid", context="talk")

# Create the plot
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))  # Adjust the figure size as needed

# Create the boxplot
sns.boxplot(data=sorted_dat, orient='h', ax=ax, palette='deep')

# Set the labels
ax.set_xlabel("ASD Bias", fontsize=16, weight='bold')
ax.set_yticklabels(sorted_CNU_HYa_Glut, fontsize=12)

# Improve grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Customize the ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
# Sample data generation (using the provided example structure)
Subclass = sorted(Subclass2Cluster.keys())
CNU_HYa_Glut = Subclass[73:91]
dat = []
for _CT in CNU_HYa_Glut:
    dat.append([x for x in ASD_BiasDN_z1clip3_addP.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])

# Calculate medians and sort data based on medians
medians = [np.median(d) for d in dat]
sorted_indices = np.argsort(medians)[::-1]
sorted_dat = [dat[i] for i in sorted_indices]
sorted_CNU_HYa_Glut = [CNU_HYa_Glut[i] for i in sorted_indices]

# Set up the plotting style and context
sns.set(style="whitegrid", context="talk")

# Create the plot
fig, ax = plt.subplots(dpi=480, figsize=(10, 6))  # Adjust the figure size as needed

# Create the boxplot
sns.boxplot(data=sorted_dat, orient='h', ax=ax, palette='deep')

# Set the labels
ax.set_xlabel("ASD Bias", fontsize=16, weight='bold')
ax.set_yticklabels(sorted_CNU_HYa_Glut, fontsize=12)

# Improve grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Customize the ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
# Sample data generation (using the provided example structure)
Subclass = sorted(Subclass2Cluster.keys())
CNU_HYa_Glut = Subclass[146:154] +  Subclass[58:65] + Subclass[36:39]
dat = []
for _CT in CNU_HYa_Glut:
    dat.append([x for x in ASD_BiasDN_z1clip3_addP.loc[Subclass2Cluster[_CT], "EFFECT"].values if x==x])

# Calculate medians and sort data based on medians
medians = [np.median(d) for d in dat]
sorted_indices = np.argsort(medians)[::-1]
sorted_dat = [dat[i] for i in sorted_indices]
sorted_CNU_HYa_Glut = [CNU_HYa_Glut[i] for i in sorted_indices]

# Set up the plotting style and context
sns.set(style="whitegrid", context="talk")

# Create the plot
fig, ax = plt.subplots(dpi=480, figsize=(8, 6))  # Adjust the figure size as needed

# Create the boxplot
sns.boxplot(data=sorted_dat, orient='h', ax=ax, palette='deep')

# Set the labels
ax.set_xlabel("ASD Bias", fontsize=16, weight='bold')
ax.set_yticklabels(sorted_CNU_HYa_Glut, fontsize=12)

# Improve grid and layout
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Customize the ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)

plt.show()

# F (STR vs CT Mat)

In [ ]:
MERFISH = pd.read_csv("dat/MERFISH/MERFISH.cells.ASD.Bias.Anno.csv", index_col=0)

In [ ]:
ASD_Circuits_cleannames = [" ".join(x.split("_")) for x in ASD_Circuits]
#ASD_Circuits_cleannames.append("Subiculum")
ASD_Circuits_cleannames.remove("Subiculum ventral part")
ASD_Circuits_cleannames.remove("Subiculum dorsal part")

In [ ]:
MERFISH_Circuit = MERFISH[MERFISH["ISH_STR"].isin(ASD_Circuits_cleannames)]
MERFISH_Circuit.shape

In [ ]:
class_counts = MERFISH_Circuit["class"].value_counts()
class_counts = class_counts[class_counts>1000]

In [ ]:
sns.set(style="whitegrid")
plt.figure(dpi=300, figsize=(6, 6))
class_counts.plot(kind='bar', color='grey')

# Adding labels and title
plt.xlabel('Cell Class', fontsize=14, weight='bold')
plt.ylabel('Number of Cells in ASD Circuit', fontsize=14, weight='bold')
#plt.title('Bar Plot from Pandas Series', fontsize=14)
plt.yscale('log')  # Set y-axis to log scale
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
ASD_Cir_Clusters = set(MERFISH_Circuit["cluster"].values)

In [ ]:
ASD_Cluster_Bias_for_DN = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.z1clip3.addP.csv", index_col=0)

In [ ]:
Cir_ASD_Cluster_Bias_for_DN_sub = ASD_Cluster_Bias_for_DN[ASD_Cluster_Bias_for_DN.index.isin(ASD_Cir_Clusters)]

In [ ]:
# Annotate ASD Bias
for i, row in MERFISH_Circuit.iterrows():
    cluster = row["cluster"]
    if cluster not in ASD_Cluster_Bias_for_DN.index.values:
        MERFISH_Circuit.loc[i, "ASD.Bias"] = 0
    else:
        MERFISH_Circuit.loc[i, "ASD.Bias"] = ASD_Cluster_Bias_for_DN.loc[cluster, "EFFECT"]
    #MERFISH_Circuit.loc[i, "SCZ.Bias.adj"] = SCZ_Cluster_Bias_for_DN.loc[cluster, "Bias_Adj"]

In [ ]:
cluster_counts = MERFISH_Circuit["cluster"].value_counts()
cluster_counts = cluster_counts[cluster_counts>50]

In [ ]:
cluster_counts_df = pd.DataFrame(cluster_counts)
cluster_counts_df.columns = ['Count']
cluster_counts_df.index.name = 'Cluster'
cluster_counts_df.to_csv("dat/ASD.Cluster.Count.csv")

In [ ]:
cluster_counts.shape

In [ ]:
subclass_counts = MERFISH_Circuit["subclass"].value_counts()
subclass_counts = subclass_counts[subclass_counts>500]

In [ ]:
asd_str_subclass_bias_dat = []
for STR_ in ASD_Circuits_cleannames:
    tmp = MERFISH_Circuit[MERFISH_Circuit["ISH_STR"]==STR_]
    _str_subclass_bias_dat = []
    for subclass,n in subclass_counts.items():
        tmptmp = tmp[tmp["subclass"]==subclass]
        if tmptmp.shape[0] == 0:
            _str_subclass_bias_dat.append(0)
        else:
            _str_subclass_bias_dat.append(np.nanmean(tmptmp["ASD.Bias"].values))
            #_str_subclass_bias_dat.append(np.nanmean(tmptmp["SCZ.Bias.1000"].values))
    asd_str_subclass_bias_dat.append(_str_subclass_bias_dat)
asd_str_subclass_bias_dat = np.array(asd_str_subclass_bias_dat)

In [ ]:
Subclass_STR_Bias_DF = pd.DataFrame(data=asd_str_subclass_bias_dat, index=ASD_Circuits_cleannames, 
                                    columns=subclass_counts.index.values)
Subclass_STR_Bias_DF = Subclass_STR_Bias_DF.transpose()

In [ ]:
subclass_dat_v1 = []
subclass_dat_v2 = []
for STR_ in ASD_Circuits_cleannames:
    tmp = MERFISH_Circuit[MERFISH_Circuit["ISH_STR"]==STR_]
    print(STR_, tmp.shape)
    str_subclass_frac_dat_1 = []
    str_subclass_frac_dat_2 = []
    for subclass_,n in subclass_counts.items():
        tmptmp = tmp[tmp["subclass"]==subclass_]
        n_cell = tmptmp.shape[0]
        frac_cell_1 = n_cell/n
        frac_cell_2 = n_cell/tmp.shape[0]
        str_subclass_frac_dat_1.append(frac_cell_1)
        str_subclass_frac_dat_2.append(frac_cell_2)
    subclass_dat_v1.append(str_subclass_frac_dat_1)
    subclass_dat_v2.append(str_subclass_frac_dat_2)
subclass_dat_v1 = np.array(subclass_dat_v1)
subclass_dat_v2 = np.array(subclass_dat_v2)

##### Get Region composition of sublcass cell types

In [ ]:
ASD_STR_Bias.head(2)

In [ ]:
MERFISH_Circuit.head(2)

##### Get STR composition of sublcass cell types and plot biases

In [ ]:
SubclassCellComp = pd.DataFrame(data=subclass_dat_v2, index=ASD_Circuits_cleannames, columns=subclass_counts.index.values)
SubclassCellComp = SubclassCellComp.transpose()
SubclassCellComp = SubclassCellComp.sort_index()

In [ ]:
Cir_ISH_Bias = ASD_STR_Bias[ASD_STR_Bias.index.isin(ASD_Circuits)]
Cir_ISH_Bias_sorted = Cir_ISH_Bias.sort_values(by=['REGION', 'EFFECT'])

In [ ]:
STR_Sort_Names = []
RegionSorts = ["Isocortex", "Olfactory_areas", "Cortical_subplate", "Hippocampus", "Striatum", 
           "Amygdala", "Pallidum", "Thalamus", "Midbrain"]
STR_Reg = {}
for REG in RegionSorts:
    tmp = Cir_ISH_Bias[Cir_ISH_Bias["REGION"]==REG]
    ShortNames = [" ".join(x.split("_")) for x in tmp.index.values]
    for XX in ShortNames:
        STR_Reg[XX] = REG
    STR_Sort_Names.extend(ShortNames)

In [ ]:
RegionSorts = ["Isocortex", "Olfactory_areas", "Cortical_subplate", "Hippocampus", "Striatum", 
           "Amygdala", "Pallidum", "Thalamus", "Midbrain"]
Colors = ["#0098D4", "#783F04", "#9FC5E8", "#6AA84F", "#E69138", "#674EA7", 
         "#674EA7", "#F44336", "#783F04"]
region_colors = dict(zip(RegionSorts, Colors))

In [ ]:
bias_df = Subclass_STR_Bias_DF

In [ ]:
SubclassCellComp.to_csv("dat/ASD.Cir.SubClassCellComp.csv")

In [ ]:
Subclass_STR_Bias_DF.to_csv("dat/ASD.Cir.SubClassBias.csv")

In [ ]:
SubclassCellComp = pd.read_csv("dat/ASD.Cir.SubClassCellComp.csv", index_col=0)
Subclass_STR_Bias_DF = pd.read_csv("dat/ASD.Cir.SubClassBias.csv", index_col=0)

In [ ]:
ASD_STR_Bias["REGION"].value_counts()

In [ ]:
x_labels = SubclassCellComp.index
y_labels = STR_Sort_Names 
y_labels = y_labels[::-1]

# # Prepare data for scatter plot
# x = []
# y = []
# sizes = []
# colors = []

# norm = plt.Normalize(-0.6, 0.6)

# sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
# sm.set_array([])

# for i, structure  in enumerate(x_labels):
#     for j, cell_type in enumerate(y_labels):
#         x.append(i)
#         y.append(j)
#         #sizes.append(SubclassCellComp.loc[structure, cell_type] * 3500)  # Scale up the sizes for better visualization
#         sizes.append(size_transform(SubclassCellComp.loc[structure, cell_type]))
#         bias_value = bias_df.loc[structure, cell_type]
#         colors.append(sm.to_rgba(bias_value))

In [ ]:
regions = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 
               'Hippocampus','Amygdala','Striatum', 
               "Thalamus", "Hypothalamus", "Midbrain", 
               "Medulla", "Pallidum", "Pons", 
               "Cerebellum"]
region_colors = dict(zip(regions, ["#268ad5", "#D5DBDB", "#7ac3fa", 
                                 "#2c9d39", "#742eb5", "#ed8921", 
                                 "#e82315", "#E6B0AA", "#f6b26b",  
                                 "#20124d", "#2ECC71", "#D2B4DE", 
                                 "#ffd966", ]))
y_region_seq = [ASD_STR_Bias.loc["_".join(x.split()), "REGION"] for x in y_labels]
def shorten_name(x):
    if x =="Lateral septal nucleus rostral rostroventral part":
        return "Lateral septal nucleus"
    else:
        return x
def size_transform(comp):
    if comp ==0:
        return 0
    else:
        return 100 + comp * 2500

In [ ]:
def size_transform(comp):
    if comp ==0:
        return 0
    else:
        return 100 + comp * 2500
comp = [0.001, 0.01, 0.1, 0.5]
size_of_comps = [size_transform(x) for x in comp]

fig,  ax = plt.subplots(dpi=480, figsize=(40, 28))

# Plot scatter points for each `comp` value with corresponding sizes
for i, (c, size) in enumerate(zip(comp, size_of_comps)):
    ax.scatter([], [], s=size, label=f'comp = {c}')

# Create the legend
ax.legend(loc='center', bbox_to_anchor=(0.5, 0.5), fontsize=12, frameon=False)

# Remove axes for a cleaner look
ax.axis('off')

# Adjust layout and show the legend
plt.tight_layout()
plt.show()

In [ ]:
x_labels = SubclassCellComp.index
y_labels = STR_Sort_Names 
y_labels = y_labels[::-1]

# Prepare data for scatter plot
x = []
y = []
sizes = []
colors = []

norm = plt.Normalize(-0.6, 0.6)

sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
sm.set_array([])

for i, structure  in enumerate(x_labels):
    for j, cell_type in enumerate(y_labels):
        x.append(i)
        y.append(j)
        #sizes.append(SubclassCellComp.loc[structure, cell_type] * 3500)  # Scale up the sizes for better visualization
        sizes.append(size_transform(SubclassCellComp.loc[structure, cell_type]))
        bias_value = bias_df.loc[structure, cell_type]
        colors.append(sm.to_rgba(bias_value))

In [ ]:
regions = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 
               'Hippocampus','Amygdala','Striatum', 
               "Thalamus", "Hypothalamus", "Midbrain", 
               "Medulla", "Pallidum", "Pons", 
               "Cerebellum"]
region_colors = dict(zip(regions, ["#268ad5", "#D5DBDB", "#7ac3fa", 
                                 "#2c9d39", "#742eb5", "#ed8921", 
                                 "#e82315", "#E6B0AA", "#f6b26b",  
                                 "#20124d", "#2ECC71", "#D2B4DE", 
                                 "#ffd966", ]))
y_region_seq = [ASD_STR_Bias.loc["_".join(x.split()), "REGION"] for x in y_labels]
def shorten_name(x):
    if x =="Lateral septal nucleus rostral rostroventral part":
        return "Lateral septal nucleus"
    else:
        return x
def size_transform(comp):
    if comp ==0:
        return 0
    else:
        return 100 + comp * 2500


In [ ]:
fig,  ax = plt.subplots(dpi=480, figsize=(50, 28))

# Scatter plot
scatter = ax.scatter(x, y, s=sizes, alpha=0.8, c=colors)

# Prepare y-labels with extra space

y_labels2 = ["{}     ".format(shorten_name(x)) for x in y_labels]
ax.set_xticks(np.arange(len(x_labels)))
ax.set_xticklabels(x_labels, rotation=90, fontsize=30)
ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels(y_labels2, fontsize=42)

# Adjust the layout to give more space for y-tick labels
plt.subplots_adjust(left=0.15, right=0.9, top=0.95, bottom=0.15)  # Increase 'left' to give more space for y-tick labels

# Add the region color column as a new axis
ax_region = fig.add_axes([0.135, 0.15, 0.01, 0.79])  # Adjust the position of the region bar

ax_region.set_yticks(np.arange(len(y_labels)))
ax_region.set_yticklabels([])  # No labels, just colors
ax_region.set_xticklabels([])
ax_region.tick_params(left=False, right=False)  # No ticks

# Color each y-tick according to its region
for i, region in enumerate(y_region_seq):
    if region == "Amygdalar":
        region = "Amygdala"
    ax_region.add_patch(plt.Rectangle((0, i - 0.5), 1, 1, color=region_colors[region]))

# Add marker size legend for cell composition (marker and font 50% larger)
ax.scatter([], [], s=0, label='Cell composition', color="grey")
for i, (c, size) in enumerate(zip(comp, size_of_comps)):
    ax.scatter([], [], s=size * 1.5, label=f' : {c}', color="grey")

# Create the legend
ax.legend(loc='center', bbox_to_anchor=(1.1, -0.1), fontsize=52, frameon=False)

# Adjust the grid and spines
ax.grid(True, linestyle='--', alpha=0.1)
ax.set_xlim(-0.5, len(x_labels))
ax.set_ylim(-0.5, len(y_labels))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Save the figure
plt.savefig("../results/figs/ASD_top60_SubClass_CellCompBias_with_regions.pdf", bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.patches as mpatches

region_colors2 = {'Isocortex': '#268ad5',
 'Olfactory areas': '#D5DBDB',
 'Cortical subplate': '#7ac3fa',
 'Hippocampus': '#2c9d39',
 'Amygdala': '#742eb5',
 'Striatum': '#ed8921',
 'Thalamus': '#e82315',
 'Midbrain': '#f6b26b',
 'Pallidum': '#2ECC71',}

# Create a figure for the legend
fig, ax = plt.subplots(dpi=360, figsize=(10, 6))

# Create patches for each region
patches = [mpatches.Patch(color=color, label=region) for region, color in region_colors2.items()]

# Add the legend to the plot and make it take up the whole figure
ax.legend(handles=patches, loc='center', ncol=2, fontsize=50, frameon=False,
          bbox_to_anchor=(0.5, 0.5), bbox_transform=plt.gcf().transFigure)

# Remove axes for a cleaner look
ax.axis('off')

# Adjust layout to remove any padding
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

# Show the legend
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=200, figsize=(35, 20))

scatter = ax.scatter(x, y, s=sizes, alpha=0.8, c=colors)

y_labels2 = ["{}    ".format(x) for x in y_labels]
ax.set_xticks(np.arange(len(x_labels)))
ax.set_xticklabels(x_labels, rotation=90, fontsize=25)
ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels(y_labels2, fontsize=30)

for tick in plt.gca().get_yticklabels():
    tick.set_verticalalignment('center')
    tick.set_multialignment('center')

ax.grid(True, linestyle='--', alpha=0.1)
ax.set_xlim(-0.5, len(x_labels))
ax.set_ylim(-0.5, len(y_labels))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Add colorbar for bias values
cbar = plt.colorbar(sm, orientation="vertical")
cbar.set_label('Bias', fontsize=25)
cbar.ax.tick_params(labelsize=25)

plt.tight_layout()
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.15)
plt.savefig("figs/ASD_top60_SubClass_CellCompBias.v2.pdf", bbox_inches='tight')

In [ ]:
region_colors

In [ ]:
def ModifySTRName(STR_Names):
    res = []
    for Name in STR_Names:
        tokens = Name.split()
        len_token = len(tokens)
        if len(Name) > 10 and len_token > 1:
            len_token = len(tokens)
            #print(tokens, len_token)
            break_point = int(len_token/2)
            line1 = tokens[:break_point]
            line2 = tokens[break_point:]
            NewName = "{}\n{}".format(" ".join(line1), " ".join(line2))
        else:
            NewName = Name
        res.append(NewName)
    return res
y_labels2 = ModifySTRName(y_labels)

In [ ]:
bias_df.head(2)

In [ ]:
dat_ct_neur_comps = []
dat_ct_neur_bias = []
dat_ct_glia_comps = []
dat_ct_glia_bias = []
for i, str in enumerate(SubclassCellComp.columns.values):
    #print(str)
    list_ct = SubclassCellComp.index.values
    NN = [x for x in list_ct if x.endswith("NN")]
    NEU = [x for x in list_ct if not x.endswith("NEU")]
    str_ct_neur_comps = SubclassCellComp.loc[NEU, str]
    str_ct_neur_bias = bias_df.loc[NEU, str]
    str_ct_glia_comps = SubclassCellComp.loc[NN, str]
    str_ct_glia_bias = bias_df.loc[NN, str]
    
    dat_ct_neur_comps.extend(str_ct_neur_comps)
    dat_ct_neur_bias.extend(str_ct_neur_bias)
    dat_ct_glia_comps.extend(str_ct_glia_comps)
    dat_ct_glia_bias.extend(str_ct_glia_bias)



In [ ]:
# Calculate Pearson correlation coefficients
r_neur, p_neur = pearsonr(dat_ct_neur_comps, dat_ct_neur_bias)
r_non_neur, p_non_neur = pearsonr(dat_ct_glia_comps, dat_ct_glia_bias)

# Create scatter plot
plt.figure(figsize=(8, 7), dpi=120)  # Increased figure size

# Plot neurons
plt.scatter(dat_ct_neur_comps, dat_ct_neur_bias, alpha=0.6, edgecolors='w', color='blue', label='Neurons', s=50)  # Increased marker size

# Plot glia
plt.scatter(dat_ct_glia_comps, dat_ct_glia_bias, alpha=0.6, edgecolors='w', color='red', label='Non-Neurons', s=50)  # Increased marker size

# Set labels and title
plt.xlabel("Cell Composition Bias", fontsize=18)  # Increased font size
plt.ylabel("Cell Type Bias", fontsize=18)  # Increased font size
plt.title("Cell Composition vs Cell Type Bias", fontsize=20)  # Increased font size

# Add correlation information to the plot
plt.text(0.55, 0.25, f"Neurons: r={r_neur:.3f}, p={p_neur:.1e}", transform=plt.gca().transAxes, fontsize=14, color='blue')  # Increased font size
plt.text(0.55, 0.20, f"Non-Neurons: r={r_non_neur:.3f}, p={p_non_neur:.1e}", transform=plt.gca().transAxes, fontsize=14, color='red')  # Increased font size

# Add legend
plt.legend(fontsize=16)  # Increased font size

# Add grid and improve layout
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Increase tick label font size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Show plot
plt.show()

In [ ]:
SubclassCellComp

# Test Connectivity

In [ ]:
# For Sibling
ScoreMatDir="/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw_v3/"
IpsiInfoMat=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.csv", index_col=0)
IpsiInfoMatShort_v1=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.Short.3900.csv", index_col=0)
IpsiInfoMatLong_v1=pd.read_csv(ScoreMatDir + "InfoMat.Ipsi.Long.3900.csv", index_col=0)

In [ ]:
def FixSubiculum_reverse(DF):
    Z = DF.loc["Subiculum"]
    DF.loc["Subiculum_dorsal_part", :] = Z
    DF.loc["Subiculum_ventral_part", :] = Z
    DF = DF.drop(["Subiculum"])
    DF = DF.sort_values("EFFECT", ascending=False)
    return DF

In [ ]:
ASD_Bias_MERFISH_STR_Z2_CM = pd.read_csv("dat/Bias/STR/ASD.MERFISH_Allen.CM.ISHMatch.Z2.csv", index_col=0)
ASD_Bias_MERFISH_STR_Z2_VM = pd.read_csv("dat/Bias/STR/ASD.MERFISH_Allen.VM.ISHMatch.Z2.csv", index_col=0)

In [ ]:
ASD_Bias_MERFISH_STR_Z2_CM = FixSubiculum_reverse(ASD_Bias_MERFISH_STR_Z2_CM)
ASD_Bias_MERFISH_STR_Z2_VM = FixSubiculum_reverse(ASD_Bias_MERFISH_STR_Z2_VM)

In [ ]:
ASD_Bias_MERFISH_STR_Z2_VM.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.VM.ISHMatch.Z2.splitSB.csv")

In [ ]:
XX = pd.read_csv("dat/Bias/STR/ASD.MERFISH_Allen.NM.ISHMatch.Z2.csv", index_col=0)
XX = ASD_Bias_MERFISH_STR_Z2_VM = FixSubiculum_reverse(XX)
XX.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.NM.ISHMatch.Z2.splitSB.csv")

In [ ]:
#ASD_159_bias = pd.read_csv("../../ASD_Circuits/dat/Unionize_bias/ASD.159.pLI.z2.csv", index_col=0)

In [ ]:
#ASD_159_bias.head(50)

In [ ]:
STR_Ranks = ASD_Bias_MERFISH_STR_Z2_CM.index.values
#STR_Ranks = ASD_Bias.index.values
topNs = list(range(200, 5, -1))
SC_Agg_topN_score = []
SC_Agg_topN_scoreShort = []
SC_Agg_topN_scoreLong = []
for topN in topNs:
    top_strs = STR_Ranks[:topN]
    score = ScoreCircuit_SI_Joint(top_strs, IpsiInfoMat)
    scoreShort = ScoreCircuit_SI_Joint(top_strs, IpsiInfoMatShort_v1)
    scoreLong = ScoreCircuit_SI_Joint(top_strs, IpsiInfoMatLong_v1)
    SC_Agg_topN_score.append(score)
    SC_Agg_topN_scoreShort.append(scoreShort)
    SC_Agg_topN_scoreLong.append(scoreLong)
SC_Agg_topN_score = np.array(SC_Agg_topN_score)
SC_Agg_topN_scoreShort = np.array(SC_Agg_topN_scoreShort)
SC_Agg_topN_scoreLong = np.array(SC_Agg_topN_scoreLong)

In [ ]:
topNs = np.arange(200, 5, -1)
DIR = "/home/jw3514/Work/ASD_Circuits/scripts/RankScores"
Cont_Distance = np.load("{}/RankScore.Ipsi.Cont.npy".format(DIR))
Cont_DistanceShort = np.load("{}/RankScore.Ipsi.Short.3900.Cont.npy".format(DIR))
Cont_DistanceLong = np.load("{}/RankScore.Ipsi.Long.3900.Cont.npy".format(DIR))

In [ ]:
for n, obs, obs1, obs2 in list(zip(topNs, SC_Agg_topN_score, SC_Agg_topN_scoreShort, SC_Agg_topN_scoreLong)):
    if n < 60:
        Null = Cont_Distance[:, np.where(topNs==n)].flatten()
        z, p = GetPermutationP(Null, obs)
        
        Null = Cont_DistanceShort[:, np.where(topNs==n)].flatten()
        z1, p1 = GetPermutationP(Null, obs1)
        
        Null = Cont_DistanceLong[:, np.where(topNs==n)].flatten()
        z2, p2 = GetPermutationP(Null, obs2)
        print("%d %.3f %.4f %.4f %.4f"%(n, obs, p, p1, p2))

In [ ]:
import matplotlib.backends.backend_pdf

fig, (ax1, ax2, ax3) = plt.subplots(3,1, dpi=480, figsize=(7,11))

BarLen = 34.1
#BarLen = 47.5

cont = np.median(Cont_Distance, axis=0)
ax1.plot(topNs, SC_Agg_topN_score, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASD\nProbands")

lower = np.percentile(Cont_Distance, 50-BarLen, axis=0)
upper = np.percentile(Cont_Distance, 50+BarLen, axis=0)
ax1.errorbar(topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
            yerr=(cont - lower, upper - cont ), ls="dashed", label="Siblings")
ax1.set_xlabel("Structure Rank\n", fontsize=17)
ax1.set_ylabel("Circuit Connectivity Score", fontsize=15)
ax1.legend(fontsize=13)

cont = np.nanmean(Cont_DistanceLong, axis=0)
ax2.plot(topNs, SC_Agg_topN_scoreLong, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASD\nProbands")

lower = np.nanpercentile(Cont_DistanceLong, 50-BarLen, axis=0)
upper = np.nanpercentile(Cont_DistanceLong, 50+BarLen, axis=0)
ax2.errorbar(topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
            yerr=(cont - lower, abs(upper - cont) ), ls="dashed", label="Siblings")
ax2.set_xlabel("Structure Rank\n", fontsize=17)
ax2.set_ylabel("Circuit Connectivity Score", fontsize=15)
ax2.legend(fontsize=13)

cont = np.median(Cont_DistanceShort, axis=0)
ax3.plot(topNs, SC_Agg_topN_scoreShort, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASD\nProbands")

lower = np.percentile(Cont_DistanceShort, 50-BarLen, axis=0)
upper = np.percentile(Cont_DistanceShort, 50+BarLen, axis=0)
ax3.errorbar(topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
            yerr=(cont - lower, upper - cont ), ls="dashed", label="Siblings")
ax3.set_xlabel("Structure Rank\n", fontsize=17)
ax3.set_ylabel("Circuit Connectivity Score", fontsize=15)
ax3.legend(fontsize=13)

#fig.text(0.5, -0.03, 'Top Number of Structuress', ha='center')
#fig.text(-0.03, 0.5, 'SI Score', va='center', rotation='vertical')
ax1.set_xlim(0, 121)
ax2.set_xlim(0, 121)
ax3.set_xlim(0, 121)
plt.tight_layout()
#plt.legend(fontsize=15)
ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
#plt.savefig("../figs/main/Fig2.BCD.pdf")
#plt.savefig("../figs/main/Fig2.BCD.png")

In [ ]:
ASD_Bias_MERFISH_STR_Z2_NM = XX

In [ ]:
STR_Ranks = ASD_Bias_MERFISH_STR_Z2_NM.index.values
#STR_Ranks = ASD_Bias.index.values
topNs = list(range(200, 5, -1))
SC_Agg_topN_score = []
SC_Agg_topN_scoreShort = []
SC_Agg_topN_scoreLong = []
for topN in topNs:
    top_strs = STR_Ranks[:topN]
    score = ScoreCircuit_SI_Joint(top_strs, IpsiInfoMat)
    scoreShort = ScoreCircuit_SI_Joint(top_strs, IpsiInfoMatShort_v1)
    scoreLong = ScoreCircuit_SI_Joint(top_strs, IpsiInfoMatLong_v1)
    SC_Agg_topN_score.append(score)
    SC_Agg_topN_scoreShort.append(scoreShort)
    SC_Agg_topN_scoreLong.append(scoreLong)
SC_Agg_topN_score = np.array(SC_Agg_topN_score)
SC_Agg_topN_scoreShort = np.array(SC_Agg_topN_scoreShort)
SC_Agg_topN_scoreLong = np.array(SC_Agg_topN_scoreLong)

In [ ]:
Cont_Distance[:, np.where(topNs==50)].flatten()

In [ ]:
for n, obs, obs1, obs2 in list(zip(topNs, SC_Agg_topN_score, SC_Agg_topN_scoreShort, SC_Agg_topN_scoreLong)):
    if n < 60:
        Null = Cont_Distance[:, np.where(topNs==n)].flatten()
        z, p = GetPermutationP(Null, obs)
        
        Null = Cont_DistanceShort[:, np.where(topNs==n)].flatten()
        z1, p1 = GetPermutationP(Null, obs1)
        
        Null = Cont_DistanceLong[:, np.where(topNs==n)].flatten()
        z2, p2 = GetPermutationP(Null, obs2)
        print("%d %.3f %.4f %.4f %.4f"%(n, obs, p, p1, p2))

In [ ]:
import matplotlib.backends.backend_pdf

fig, (ax1, ax2, ax3) = plt.subplots(3,1, dpi=480, figsize=(7,11))

BarLen = 34.1
#BarLen = 47.5

cont = np.median(Cont_Distance, axis=0)
ax1.plot(topNs, SC_Agg_topN_score, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASD\nProbands")

lower = np.percentile(Cont_Distance, 50-BarLen, axis=0)
upper = np.percentile(Cont_Distance, 50+BarLen, axis=0)
ax1.errorbar(topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
            yerr=(cont - lower, upper - cont ), ls="dashed", label="Siblings")
ax1.set_xlabel("Structure Rank\n", fontsize=17)
ax1.set_ylabel("Circuit Connectivity Score", fontsize=15)
ax1.legend(fontsize=13)

cont = np.nanmean(Cont_DistanceLong, axis=0)
ax2.plot(topNs, SC_Agg_topN_scoreLong, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASD\nProbands")

lower = np.nanpercentile(Cont_DistanceLong, 50-BarLen, axis=0)
upper = np.nanpercentile(Cont_DistanceLong, 50+BarLen, axis=0)
ax2.errorbar(topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
            yerr=(cont - lower, abs(upper - cont) ), ls="dashed", label="Siblings")
ax2.set_xlabel("Structure Rank\n", fontsize=17)
ax2.set_ylabel("Circuit Connectivity Score", fontsize=15)
ax2.legend(fontsize=13)

cont = np.median(Cont_DistanceShort, axis=0)
ax3.plot(topNs, SC_Agg_topN_scoreShort, color="blue", marker="o", markersize=5, lw=1,
                     ls="dashed", label="ASD\nProbands")

lower = np.percentile(Cont_DistanceShort, 50-BarLen, axis=0)
upper = np.percentile(Cont_DistanceShort, 50+BarLen, axis=0)
ax3.errorbar(topNs, cont, color="grey", marker="o", markersize=1.5, lw=1,
            yerr=(cont - lower, upper - cont ), ls="dashed", label="Siblings")
ax3.set_xlabel("Structure Rank\n", fontsize=17)
ax3.set_ylabel("Circuit Connectivity Score", fontsize=15)
ax3.legend(fontsize=13)

#fig.text(0.5, -0.03, 'Top Number of Structuress', ha='center')
#fig.text(-0.03, 0.5, 'SI Score', va='center', rotation='vertical')
ax1.set_xlim(0, 121)
ax2.set_xlim(0, 121)
ax3.set_xlim(0, 121)
plt.tight_layout()
#plt.legend(fontsize=15)
ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
#plt.savefig("../figs/main/Fig2.BCD.pdf")
#plt.savefig("../figs/main/Fig2.BCD.png")

In [ ]:
XX = pd.read_csv("dat/JonDat/MERFISH_Bias_NeuroOnly_JC.csv", index_col=0)
XX = ASD_Bias_MERFISH_STR_Z2_VM = FixSubiculum_reverse(XX)
XX.to_csv("dat/JonDat/MERFISH_Bias_NeuroOnly_JC.splitSB.csv")